In [ ]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading

seed = 46

world = Village()
village_side = 39
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15, seed=seed)
#vgraph_builder = world.to_drake_plant()
print(len(world.obstacles))

In [ ]:
import pickle
from pydrake.all import HPolyhedron, VPolytope


regions = []
with open('experiment_village_greedy_39_10_09_27_04_59_24___46_500_0.050_0.200/data/it_6.pkl', 'rb') as f:
    data= pickle.load(f)

for rga, rgb in zip(data['ra'], data['rb']):
    for ra, rb in zip(rga, rgb):
        regions.append(HPolyhedron(ra, rb))

In [ ]:
intersections = []

for i in range(len(regions)):
    for j in range(i+1, len(regions)):
        if regions[i].IntersectsWith(regions[j]):
            intersections.append(regions[i].Intersection(regions[j]))
print(intersections[0])
print(len(intersections))

# keep things clean, as simple as possible, don´t focus on speed yet
# check for redundancy = yes
# And reduce inequalities
# use networkx A star
# use networkx's existing methods to ask it for an adjacency matrix, for A star, put it into this graph form FIRST

# Plotting Utils

In [ ]:
import os
import mcubes
from meshcat.geometry import TriangularMeshGeometry, MeshLambertMaterial
from matplotlib.colors import to_hex as _to_hex

to_hex = lambda rgb: '0x' + _to_hex(rgb)[1:]
def col_hand(q1,q2,q3):
    p =np.array([q1,q2,q3])
    # if np.any(world.L_dom>p):
    #     return 1
    # if np.any(world.U_dom<p):
    #     return 1
    return 1.0*world.col_handle(np.array([q1,q2,q3]))

def col_hand_neg(q1,q2,q3):
    p =np.array([q1,q2,q3])
    if np.any(world.L_dom>p):
        return 0
    if np.any(world.U_dom<p):
        return 0
    return 1- 1.0*world.col_handle(p)

def plot_collision_constraint(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand, 0.5)
        with open(f"tmp/col_cons{N}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col']
    color = to_hex((1,0,0))
    material = MeshLambertMaterial(color=color, opacity=0.4,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)

def plot_collision_constraint_neg(N = 50, q_min = np.array(world.L_dom)-5, q_max= np.array(world.U_dom)+5):
    if f"col_cons{N}_n.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons{N}_n.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min), tuple(
                q_max), N, N, N, col_hand_neg, 0.5)
        with open(f"tmp/col_cons{N}_n.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    #tri_drake = [SurfaceTriangle(*t) for t in triangles]

    #vertices += _offset_meshcat_2.reshape(-1,3)
    obj = world['col_n']
    color = to_hex((0,1,0))
    material = MeshLambertMaterial(color=color, opacity=0.3,  wireframe=False)
    obj.set_object(TriangularMeshGeometry(vertices, triangles),
                                   material)
from meshcat.geometry import Cylinder
import meshcat.transformations as tf
from meshcat.geometry import MeshLambertMaterial

def compute_rotation_matrix(a, b):
    # Normalize the points
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    
    # Calculate the rotation axis
    rotation_axis = np.cross(a, b)
    rotation_axis /= np.linalg.norm(rotation_axis)
    
    # Calculate the rotation angle
    dot_product = np.dot(a, b)
    rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    # Construct the rotation matrix using Rodrigues' formula
    skew_matrix = np.array([[0, -rotation_axis[2], rotation_axis[1]],
                            [rotation_axis[2], 0, -rotation_axis[0]],
                            [-rotation_axis[1], rotation_axis[0], 0]])
    rotation_matrix = np.eye(3) + np.sin(rotation_angle) * skew_matrix + (1 - np.cos(rotation_angle)) * np.dot(skew_matrix, skew_matrix)
    
    return rotation_matrix

def plot_edge(pt1, pt2, name, color, opacity, size = 0.01):
    
    material = MeshLambertMaterial(color=to_hex(color), opacity=opacity)
    world[name].set_object(
                        Cylinder(np.linalg.norm(pt1-pt2), size),
                        material)
    
    dir = pt2-pt1
    rot = compute_rotation_matrix(np.array([0,1,0]), dir )
    #print(rot)
    offs = rot@np.array([0,np.linalg.norm(pt1-pt2)/2,0])
    tfmat = np.zeros((4,4))
    tfmat[:3,:3] = rot
    tfmat[:3,-1] = offs + pt1.squeeze()
    tfmat[3,3] = 1
    world[name].set_transform(tfmat)
    
def plot_edges(edges, name, color, opacity  = 1, size = 0.01):
    for i, e in enumerate(edges):
         plot_edge(e[0], e[1], name + f"/e_{i}", color, opacity, size=size)

#plot_edge(np.zeros(3), 5*np.ones(3), 'test', (1,0,0), 1, 0.8)

In [ ]:
world.plot_regions(regions, opacity=0.6)
from utils import plot_regions
 #this is for debugging the meldis meshcat -> activate kproximity to see the world
plot_regions(world.meshcat_drake, regions)

# Dummy Pathplanning and plotting of the resulting trajectory

In [ ]:
from dijkstraspp import DijkstraSPPsolver

def conv_dummy(q):
    return q

dspp = DijkstraSPPsolver(regions, conv_dummy)

In [ ]:
from meshcat.geometry import Sphere
point_in_regions(np.array([0,0, 0.5]), regions)
point_in_regions(np.array([38,38, 6.5]), regions)
world['start'].set_object(Sphere(0.6), MeshLambertMaterial(color=to_hex([0,1,0]), opacity=0.5))
world['start'].set_transform(tf.translation_matrix(np.array([0,0, 0.5])))
world['goal'].set_object(Sphere(0.6), MeshLambertMaterial(color=to_hex([0,1,0]), opacity=0.5))
world['goal'].set_transform(tf.translation_matrix(np.array([38,38, 6.5])))

In [ ]:
wps, dist = dspp.solve(np.array([0,0, 0.5]), np.array([38,38, 6.5]), refine_path=True)


In [ ]:
edges = []
for idx, wp in enumerate(wps[:-1]):
    edges.append([wp, wps[idx+1]])

plot_edges(edges, 'traj', (1,0,1), 1, 0.1 )

In [ ]:
from meshcat.transformations import translation_matrix, rotation_matrix
sign_size = [20.4, 20.2]  # Width and height of the sign
sign_position = [-20.0, 0.0, 10.0]  # X, Y, Z position of the sign
sign_color = [0.7, 0.7, 0.7]  # RGB color for the sign
from meshcat.geometry import MeshLambertMaterial, ImageTexture, Box
from PIL import Image
import meshcat
# Add the sign as a rectangle

# Load an image to put on the sign
image_path = "village_sign.png"

# Create a textured material for the sign
texture = ImageTexture(image = meshcat.geometry.PngImage.from_file(image_path))
textured_material = MeshLambertMaterial(map=texture)
sign = Box((sign_size[0], sign_size[1], 0.1))
world["sign"].set_object(sign, textured_material)
transl = translation_matrix(sign_position)
rot = rotation_matrix(np.pi/2, [1,0,0])
rot = rot@rotation_matrix(np.pi/2, [0,1,0])
rot[:,3] = transl[:,3]
world["sign"].set_transform(rot)



In [ ]:
# this saves the result to a html which you can share
# file = world.static_html()
# with open('Village.html', 'w') as f:
#     f.write(file)

In [ ]:
# if f"col_cons{100}_n.pkl" in os.listdir('tmp'):
#     with open(f"tmp/col_cons{100}_n.pkl", 'rb') as f:
#         d = pickle.load(f)
#         vertices = d['vertices']
#         triangles = d['triangles']

# output_file_path = "output_mesh.obj"

# # Write the mesh data to the .obj file
# with open(output_file_path, 'w') as obj_file:
#     obj_file.write("# Triangle Mesh\n")
    
#     # Write vertex coordinates
#     for vertex in vertices:
#         vertex = vertex
#         obj_file.write(f"v {' '.join(map(str, vertex))}\n")
    
#     # Write triangle definitions
#     for triangle in triangles:
#         # Add 1 to vertex indices because .obj files use 1-based indexing
#         obj_file.write(f"f {' '.join(map(lambda x: str(x + 1), triangle))}\n")

# print(f"Mesh saved to {output_file_path}")

# Generate new Regions (for a smaller example, see village side length)

In [ ]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading
from utils import plot_regions
from pydrake.all import Rgba

import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading


seed = 1
eps = 0.35 #this is 1- coverage -> if you want 90 coverage then set eps to 0.1
approach = 1 #leave this at 1, you will need gurobi for this
N = 100
ap_names = ['redu', 'greedy', 'nx', 'greedy_edge_CC', 'greedy_cvx_hull_fill']


world = Village()
village_side = 14 #change village side here
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15, seed=seed)
# vgraph_builder = world.to_drake_plant()
print(len(world.obstacles))


def sample_cfree_handle(n, m, regions=None):
    points = np.zeros((n,3))
    m = m*100
    if regions is None: regions = []		
    for i in range(n):
        bt_tries = 0
        while bt_tries<m:
            point = world.sample_cfree(1)[0]
            #point = world.sample_cfree(1)[0]
            if point_near_regions(point, regions, tries = 5, eps = 0.1):
                bt_tries+=1
            else:
                break
        # if bt_tries == m:
        #     return points, True
        points[i] = point
    return points, False

def estimate_coverage(regions):
    n_s = 5000
    samples = world.sample_cfree(n_s)
    in_s = 0
    for s in samples:
        if point_in_regions(s, regions):
            in_s+=1
    return (1.0*in_s)/n_s

from vislogging import LoggerClique3D
from visibility_clique_decomposition import VisCliqueDecomp
from region_generation import generate_regions_ellipses_multi_threading
from time import strftime,gmtime
from functools import partial


loggerccd = LoggerClique3D(world, f"village_{ap_names[approach]}_{village_side}_{village_height}_"+strftime("%m_%d_%H_%M_%S_", gmtime())+"_", seed, N, 1, eps, estimate_coverage)
def iris_ellipse_w_obstacles_handle(points, ellipsoids, old_regions = None):
    if len(points)>=1:
        #+ region_obstacles
        obstacles = [r for r in world.obstacles]
        regions, _, is_full = generate_regions_ellipses_multi_threading(points, ellipsoids, obstacles, world.iris_domain, estimate_coverage, coverage_threshold=1-eps, old_regs = old_regions, maxiters=1)
    return regions, is_full

VCD = VisCliqueDecomp(  N = N,
                        eps = eps,
                        max_iterations=300,
                        sample_cfree=sample_cfree_handle,
                        col_handle= world.col_handle,
                        build_vgraph= world.vgraph_handle,
                        iris_w_obstacles= iris_ellipse_w_obstacles_handle,
                        verbose=True,
                        logger=loggerccd, 
                        approach = approach,
                    )

VCD.run()

world.plot_regions(VCD.regions, opacity=0.25)
plot_regions(world.meshcat_drake, VCD.regions)

In [ ]:
import numpy as np
import networkx as nx
import math
from pydrake.all import MathematicalProgram, Solve, HPolyhedron, VPolytope, Evaluate, eq, BezierCurve_, Expression, Rgba
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


In [ ]:
# # Run this cell for the dummy case
# regions = []
# regions.append(HPolyhedron.MakeBox((0, 0), (5, 3))) # Box 0
# regions.append(HPolyhedron.MakeBox((3, 2), (8, 7))) # Box 1
# regions.append(HPolyhedron.MakeBox((6, 3), (9, 9))) # Box 2
# regions.append(HPolyhedron.MakeBox((7, 1), (11, 4))) # Box 3
# regions.append(HPolyhedron.MakeBox((10, 3), (14, 9))) # Box 4

In [ ]:
def plot_boxes(ax, boxes, facecolor='#c9daf8', faceopacity=0.5, edgecolor='#4a86e8'):
    ax.set(xlim=(0, 14), ylim=(0, 14))
    ax.set_aspect('equal', adjustable='box')
    for bl, tr in boxes:
        x1, y1 = bl
        x2, y2 = tr
        ax.add_patch(Rectangle(bl, x2-x1, y2-y1, fill=True, facecolor=facecolor, alpha=faceopacity)) 
    for bl, tr in boxes:
        x1, y1 = bl
        x2, y2 = tr
        ax.add_patch(Rectangle(bl, x2-x1, y2-y1, fill=True, facecolor='none', edgecolor=edgecolor, alpha=0.75)) 


In [ ]:
### Audrey's SuperUROP Poster Dummy Case (2D)

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
 
fig, ax = plt.subplots()

boxes_for_plotting = [#[(0, 0), (9, 2)], # wHY DOES ADDING THIS MAKE IT FAIL???
                      [(0, 8), (7.5, 10.5)],
                      [(2, 3.5), (5.5, 5)],
                      [(3, 0), (6, 13)],
                      [(5.5, 9.5), (9.5, 13.5)],
                      [(8, 0), (10, 10.5)],
                      [(8, 11), (11.5, 13)],
                      [(10.5, 1), (12, 12.5)]]

plot_boxes(ax, boxes_for_plotting)
plt.plot(4, 1, color="#e10000", marker="o", markersize=8)
plt.plot(11, 2, color="#00d100", marker="o", markersize=8)
# plt.show()
plt.savefig('/home/audurop/Desktop/initial_graph.png')

regions = []
for bl, tr in boxes_for_plotting:
    regions.append(HPolyhedron.MakeBox(bl, tr))    

In [ ]:
# regions = VCD.regions
print("#regions: ", len(regions))
region_adjacency = {i: {} for i in range(len(regions))}
intersections = {}

G = nx.Graph()

for i, region in enumerate(regions):
    for j in range(i+1, len(regions)):
        if region.IntersectsWith(regions[j]):
            intersection = regions[i].Intersection(regions[j], check_for_redundancy=True)
            G.add_node(intersection)
            intersections[intersection] = (i, j)
            region_adjacency[i][j] = intersection
            region_adjacency[j][i] = intersection

for intersection, intersection_between in intersections.items():
    for in_i in (region_adjacency[intersection_between[0]].values()):
        if (in_i != intersection):
            G.add_edge(intersection, in_i)

    for in_i in (region_adjacency[intersection_between[1]].values()):
        if (in_i != intersection):
            G.add_edge(intersection, in_i)

intersection_adjacency = nx.adjacency_matrix(G).todense()
N_intersections = len(intersections.keys())
N_regions = len(regions)
N_dims = regions[0].ambient_dimension()

print(intersection_adjacency)

In [ ]:
# 1. Define an instance of MathematicalProgram
prog = MathematicalProgram()

# 2. Add decision varaibles
x = prog.NewContinuousVariables(N_intersections, N_dims)

pairwise_dists = []
for i in range(N_intersections-1):
    for j in range(i+1, N_intersections):
        if intersection_adjacency[i][j]:
            prog.Add2NormSquaredCost(np.hstack([np.eye(N_dims, dtype=float), -np.eye(N_dims, dtype=float)]), np.zeros(N_dims), x[[i, j], :].flatten())

# 4. Add Constraints
for i, intersection in enumerate(intersections.keys()):
    var, constraints = intersection.AddPointInSetConstraints(prog, x[i])


result = Solve(prog)
print(result.get_solver_id().name())

print(len(prog.GetAllConstraints()))
print(len(result.GetInfeasibleConstraints(prog)))


# 6. Get the solution
soln = None
if result.is_success():
    soln = result.GetSolution()
    soln = np.asarray(soln).reshape(N_dims, N_intersections).T

print(soln)

weighted_intersection_adjacency = np.zeros((N_intersections, N_intersections))
weighted_edges = []
intersect = list(intersections.keys())
for i in range(N_intersections-1):
    for j in range(i+1, N_intersections):
        if intersection_adjacency[i][j]:
            weight = np.linalg.norm(soln[i] - soln[j])
            weighted_edges.append((intersect[i], intersect[j], np.linalg.norm(weight)))
            weighted_intersection_adjacency[i][j] = np.linalg.norm(weight)
            weighted_intersection_adjacency[j][i] = np.linalg.norm(weight)

G.add_weighted_edges_from(weighted_edges)

In [ ]:
start_node = (4,1) #(13, 0, 0)
end_node = (11, 2) #(0, 12, 9)

start_region = None
end_region = None
for i in range(len(regions)):
    if regions[i].PointInSet(start_node): start_region = i
    if regions[i].PointInSet(end_node): end_region = i

G.add_node(start_node)
G.add_node(end_node)
for intersection in region_adjacency[start_region].values():
    G.add_edge(start_node, intersection)
for intersection in region_adjacency[end_region].values():
    G.add_edge(end_node, intersection)
    
# pos=nx.spring_layout(G) # pos = nx.nx_agraph.graphviz_layout(G)
# nx.draw_networkx(G,pos)
# labels = nx.get_edge_attributes(G,'weight')
# nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

In [ ]:
path = nx.astar_path(G, start_node, end_node)
print(path)
print(len(path))

In [ ]:
### For Audrey's SuperUROP Figures:

fig, ax = plt.subplots()

path_in_vertices = [path[0]]
path_boxes = []

intersection_ordering = list(G.nodes())
for region in path[1:-1]:
    i = intersection_ordering.index(region)
    path_in_vertices.append(list(soln[i]))
    path_boxes.append(intersections[list(intersections.keys())[i]])

print(path_boxes)

path_in_vertices.append(path[-1])
path_in_vertices = np.array(path_in_vertices)

print(path_in_vertices)

plot_boxes(ax, boxes_for_plotting)
# plot_boxes(ax, [[(3,8), (6, 10.5)], [(8, 11), (10, 13)], [(10.5, 11), (11.5, 12.5)]], facecolor="#00d100", faceopacity=0.25, edgecolor="#00d100")
plt.plot(path_in_vertices[:, 0], path_in_vertices[:, 1], color="k", marker="o", markersize=8)
plt.plot(path_in_vertices[0, 0], path_in_vertices[0, 1], color="#e10000", marker="o", markersize=8)
plt.plot(path_in_vertices[-1, 0], path_in_vertices[-1, 1], color="#00d100", marker="o", markersize=8)
# plt.savefig('/home/audurop/Desktop/initial_plot_intersections_not_highlighted.png')

In [ ]:
def shorten_path(path):
    # 1. Define an instance of MathematicalProgram
    prog = MathematicalProgram()

    # 2. Add decision varaibles
    x = prog.NewContinuousVariables(len(path), N_dims)

    pairwise_dists = []
    for i in range(len(path)-1):
        prog.Add2NormSquaredCost(np.hstack([np.eye(N_dims), -np.eye(N_dims)]), np.zeros(N_dims), x[i:i+2].flatten())

    # # 4. Add Constraints
    prog.AddConstraint(eq(x[0], path[0]))
    prog.AddConstraint(eq(x[-1], path[-1]))
    for i in range(1, len(path)-1):
        # print(path[i])
        path[i].AddPointInSetConstraints(prog, x[i])

    # print(prog)

    # 5. Solve the problem
    result = Solve(prog)
    print(result.get_solver_id().name())

    # 6. Get the solution
    nodes = None
    if result.is_success():
        # print("shorter path found!")
        nodes = result.GetSolution()
        nodes = np.asarray(nodes).reshape(N_dims, len(path)).T

    return nodes

In [ ]:
def add_a_box(nodes, path):
    path_length = len(nodes)
    intersections_path = path
    final_path = [start_node]
    final_nodes = [start_node]
    for j in range(1, path_length-1):
        a = nodes[j-1]
        z = nodes[j]
        b = nodes[j+1]

        # Solve minimal distance without adding a box
        # 1. Define an instance of MathematicalProgram
        prog = MathematicalProgram()

        # 2. Add decision variables
        z_nobox = prog.NewContinuousVariables(N_dims)

        # 3. Add Cost function
        prog.Add2NormSquaredCost(np.eye(N_dims), -a, z_nobox.flatten())
        prog.Add2NormSquaredCost(np.eye(N_dims), -b, z_nobox.flatten())

        # Add Constraints
        intersections_path[j].AddPointInSetConstraints(prog, z_nobox)

        # 5. Solve the problem
        result = Solve(prog)
        print(result.get_solver_id().name())

        # 6. Get the solution
        if result.is_success():
            node_nobox = result.GetSolution()
            min_nobox = result.get_optimal_cost()

        # Find potential intersections
        box_a, box_b = intersections[intersections_path[j]]
        # print("checking intersection between", box_a, box_b)
        shared_boxes = set(region_adjacency[box_a].keys()).intersection(set(region_adjacency[box_b].keys()))
        # print("shared boxes", shared_boxes)
        shared_intersections = [(region_adjacency[box_a][i], region_adjacency[box_b][i]) for i in shared_boxes]
        # print(shared_intersections)

        # # Solve minimal distance WITH adding a box
        # # Loop through potential intersections, solve mathematical program for each
        min_box = np.inf
        for box_za, box_zb in shared_intersections:# union intersections[j]
            if box_za not in path and box_zb not in path:
            # 1. Define an instance of MathematicalProgram
                prog = MathematicalProgram()

                # 2. Add decision variables
                z_box = prog.NewContinuousVariables(2, N_dims)
                z_box_1 = z_box[0]
                z_box_2 = z_box[1]

                # 3. Add Cost function
                prog.Add2NormSquaredCost(np.eye(N_dims), -a, z_box_1.flatten())
                prog.Add2NormSquaredCost(np.eye(N_dims), -b, z_box_2.flatten())

                # Add Constraints
                box_za.AddPointInSetConstraints(prog, z_box_1)
                box_zb.AddPointInSetConstraints(prog, z_box_2)

                # 5. Solve the problem
                result = Solve(prog)
                print(result.get_solver_id().name())
                
                # 6. Get the solution
                if result.is_success() and result.get_optimal_cost() < min_box:
                    nodes_box = np.array(result.GetSolution()).reshape(N_dims, 2).T
                    node_box_a = nodes_box[0, :]
                    node_box_b = nodes_box[1, :]
                    min_box = result.get_optimal_cost()

        if min_box < min_nobox:
            # print("add box")
            final_path.append(box_za)
            final_path.append(box_zb)
            final_nodes.append(tuple(node_box_a))
            final_nodes.append(tuple(node_box_b))
        else:
            # print("no add box")
            final_path.append(intersections_path[j])
            final_nodes.append(tuple(node_nobox))
            
    final_nodes.append(end_node)
    final_path.append(end_node)
    return final_nodes, final_path

In [ ]:
prev_path = path
for iter in range(1):
    nodes = shorten_path(path)
    nodes, path = add_a_box(nodes, path)

    if (path == prev_path):
        print(f"path found in {iter+1} iterations")
        break
    prev_path = path
    
nodes = shorten_path(path)

In [ ]:
### For Audrey's SuperUROP Figures:

fig, ax = plt.subplots()

print(nodes)
print(path)
### Audrey's SuperUROP Poster Dummy Case (2D)

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
 
fig, ax = plt.subplots()

boxes_for_plotting = [#[(0, 0), (9, 2)], # wHY DOES ADDING THIS MAKE IT FAIL???
                      [(0, 8), (7.5, 10.5)],
                      [(2, 3.5), (5.5, 5)],
                      [(3, 0), (6, 13)],
                      [(5.5, 9.5), (9.5, 13.5)],
                      [(8, 0), (10, 10.5)],
                      [(8, 11), (11.5, 13)],
                      [(10.5, 1), (12, 12.5)]]

plot_boxes(ax, boxes_for_plotting)
plt.plot(4, 1, color="#e10000", marker="o", markersize=8)
plt.plot(11, 2, color="#00d100", marker="o", markersize=8)
# plt.show()
plt.savefig('/home/audurop/Desktop/initial_graph.png')

regions = []
for bl, tr in boxes_for_plotting:
    regions.append(HPolyhedron.MakeBox(bl, tr))    
path_boxes = []
for region in path[1:-1]:
    path_boxes.append(intersections[region])
print(path_boxes)

nodes = np.array(nodes)


plot_boxes(ax, boxes_for_plotting)
# plot_boxes(ax, [[(3,8), (6, 10.5)], [(5, 9.5), (7.5, 10.5)], [(8, 11), (10, 13)], [(10.5, 11), (11.5, 12.5)]], facecolor="#00d100", faceopacity=0.25, edgecolor="#00d100")
plt.plot(nodes[:, 0], nodes[:, 1], color="k", marker="o", markersize=8)
plt.plot(nodes[0, 0], nodes[0, 1], color="#e10000", marker="o", markersize=8)
plt.plot(nodes[-1, 0], nodes[-1, 1], color="#00d100", marker="o", markersize=8)
# plt.savefig('/home/audurop/Desktop/shortened_plot_no_highlights.png')

In [ ]:
### Audrey's SuperUROP Poster Dummy Case (3D) Figures
 
path = np.array(nodes)
world.plot_lines("/Scene/meshcat/fpp/path", path[:-1], path[1:])

In [ ]:
# Smoothing Phase
final_path = path
N = len(final_path) - 1 ## number of Bezier curves
D = 3 ## Somewhat arbitrary for now
M = 2*D + 1

boxes = []
for i in range(len(regions)):
    if regions[i].PointInSet(start_node): 
        boxes.append(start_region)
        break
for region in path[1:-1]:
    b1, b2 = intersections[region]
    if boxes[-1] != b1:
        boxes.append(b1)
    else:
        boxes.append(b2)
boxes = [regions[i] for i in boxes]

In [ ]:
def GetInfeasibleConstraintsByDualVariables(result,prog:MathematicalProgram, zero_tol = 1e-7):
    ret = []
    for c in prog.GetAllConstraints():
        duals = result.GetDualSolution(c)
        if np.any(np.abs(duals) > zero_tol):
            ret.append((c, duals))
    return ret

In [ ]:
### Projection Problem
prog = MathematicalProgram()

# 2. Add decision variables
T = np.ones((N,)) # Fixed (arbitrary) times
c_points = prog.NewContinuousVariables(M*N*N_dims) # M control points for each of the N curves
c_points = np.array(c_points).reshape(N, M, N_dims)

# Create Bezier Curves
curves = []
derivatives = [c_points] 

for i in range(D-1):
    derivatives.append([])
for i in range(N):
    curves.append(BezierCurve_[Expression](i, i+1, c_points[i].T))
    for j in range(1, D):
        derivatives[j].append((c_points[i].T * curves[i].AsLinearInControlPoints(j).todense()).T)

# print(len(curves))
# print(np.array(derivatives[0]).shape)
# print(np.array(derivatives[1]).shape)
# print(np.array(derivatives[2]).shape)

# 3. Add Cost function
# Sum (1 to D) of alpha_i * Sum (1 to N) of J(i, j)
# Calculate Q
alpha = np.ones((D,))
for j in range(N):
    Q_sum = 0
    for m in range(M):
        for n in range(M):
            Q_sum += math.comb(M, m) * math.comb(M, n) / math.comb(2*M, m+n) * np.dot(c_points[j][m], c_points[j][n])
    Q = (1 / (2 * M + 1)) * Q_sum
    for i in range(D):
        J_ij = T[j] * Q
        prog.AddCost(alpha[i] * J_ij)

# Add constraints
# Constraint 16, first and last control points are on path
con = prog.AddConstraint(eq(final_path[0], c_points[0, 0, :]))
con.evaluator().set_description("16 initial")
prog.AddConstraint(eq(final_path[-1], c_points[-1, -1, :]))
con.evaluator().set_description("16 final")


for i in range(N-1): # Constraint 17 
    for n in range(M):
        boxes[i].AddPointInSetConstraints(prog, c_points[i][n])

# Constraint 18
for j in range(N):
    for i in range(1, D):
        for n in range(M-i):
            exp = ((M - i + 1) / T[j]) * (derivatives[i-1][j][n+1] - derivatives[i-1][j][n])
            con = prog.AddConstraint(eq(derivatives[i][j][n], exp))
            con.evaluator().set_description(f"18 j={j}, i={i}, n={n}")


# Constraint 19
for i in range(D):
    for j in range(N-1):
        p_j = derivatives[i][j][M-1-i]
        p_j1 = derivatives[i][j+1][0]
        con = prog.AddConstraint(eq(p_j, p_j1))
        con.evaluator().set_description(f"19 j={j}, i={i}")


# 5. Solve the problem
result = Solve(prog)

print(len(prog.GetAllConstraints()))
print(result.get_solver_id().name())

soln = None
# 6. Get the solution
if result.is_success():
    soln = np.array(result.GetSolution()).reshape(N, M, N_dims).T
else:
    for i in GetInfeasibleConstraintsByDualVariables(result, prog):
        print(i)

# print(result.get_solution_result())
# GetExpression, PiecewisePolynomial Class (each column in one bezier curve)
# And then can plot using visualization code in slack